## Custom Packages

In [1]:
import os, sys

processing_source_path = os.path.abspath('./../../../Processing/')
if(processing_source_path not in sys.path):
    sys.path.append(processing_source_path)
from processing import preprocessing
from processing import posprocessing
from processing import FeaturesExtract

Utils_source_path = os.path.abspath('./../../../Utils/')
if(Utils_source_path not in sys.path):
    sys.path.append(Utils_source_path)
from log_soak_test import soak_test

Embedded_source_path = os.path.abspath('./../../../Quantization/functions/')
if(Embedded_source_path not in sys.path):
    sys.path.append(Embedded_source_path)
from Embedded_Model import Embedded_Model

## Python Packages

In [2]:
#protocol communication liraries
import zmq
import logging
LOG_LEVEL=logging.DEBUG #change to logging.DEBUG to enable print logs
ZEROMQ_SOCKET="tcp://127.0.0.1:12345"

import numpy as np
import time
import pickle
import matplotlib.pyplot as plt

##  EDGE AI Deep learning model

#### Signal Processing functions

In [3]:
#  EDGE AI pre-definitions
pp = preprocessing(axis=1)
ppos=posprocessing()

#### Params and initial conditions

In [4]:
params = {
    'dataset_path': '/root/data/data/OpenBMI/',
    'channels': ['O1', 'Oz', 'O2', 'PO10', 'PO9', 'POz', 'PO3', 'PO4'],
    'freqs': [0, 12.0, 8.57, 6.67, 5.45],
    'num_harmonics': 3,
    'extracted_frequencies': [4,44],
    'sfreq': 1000,
    'features_type': 'psd',
    'preprocessing': 'log10',
    'num_channels': 8,
    'num_classes':  5,
    'num_samples': 16,
    'window_size': 512,
    'step': 64,
    'num_voting': 5,
    'model_name': 'Conv2D_DW_GAP',
    'norm_axis': 3
}

window_EEG= np.random.uniform(0.00001, 0,size=(params['num_channels'],params['window_size']))
prep_time=[]
AI_time=[]
y_AI_array = np.zeros(params['num_voting'])

psd_2D = np.zeros((params['num_samples'],
                           params['extracted_frequencies'][1]-params['extracted_frequencies'][0],
                           params['num_channels']),dtype=float)

#### Load the ML model

In [5]:
#AI_tf_model=Embedded_Model('./../../keras_quantized_model/Conv2D_DW_GAP')
AI_tf_model=Embedded_Model('./../../keras_quantized_model/S1_21_Adam0')



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


#### Soak test log settings 

#### Communication protocol settings

In [6]:
context = zmq.Context()
subscriber = context.socket(zmq.SUB)
subscriber.connect(ZEROMQ_SOCKET)
subscriber.setsockopt(zmq.SUBSCRIBE, b"")

### Main-loop 

#### Soakt test time settings


In [7]:
#Acc variables
N_correct=0
N_interactions=0

# log updates time
log_update_time= 60 #seconds

# soak test total time
duration_time= 1 # hour
end_time= time.time()+ 60 * 60 * duration_time

update_time_duration=time.time()

In [8]:
log_name=f'logs/system_monitor_deep_embedded_model_2D_during_{duration_time}_hours_with_noise'
ST=soak_test(log_name=log_name)

#### AI model in loop

#### custom funcitons to 2D deep model 

In [9]:
import scipy.io
from scipy import signal

def buffer_data(data, new_batch):
    data = np.roll(data,-new_batch.shape[0],axis=0)
    data[-new_batch.shape[0]:,:] = new_batch
    
    return data

def buffer_psd(spectra, x_vector):
    spectra = np.roll(spectra, -1, axis=0)
    spectra[-1,:,:] = x_vector
    
    return spectra

# calculate psd
def calculate_psd(data, params):
    
    # print('fft inputs', eeg.shape,rep,ch,nsteps,WinLength)
    
    myamp = np.array([])

    (f, S)= scipy.signal.welch(data, params['sfreq'], nperseg=None, window = 'box', scaling='spectrum', nfft=int(params['sfreq']), average='median')

    amp = np.abs(S)
    amp = np.log10(amp[ params['extracted_frequencies'][0]:params['extracted_frequencies'][1]] + sys.float_info.epsilon)
        
    return amp


In [10]:
# noise
def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)

def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real

def create_noise(minimum, maximum,data_len):
    x = band_limited_noise(10, 900, data_len, 2048)
    x -= np.min(x)
    x /= (np.max(x))
    x *= (maximum-minimum)
    x += minimum
    return x
max_var=15/10

#### Loop

In [ ]:
#while True:
Data=[]
all_labels=[]
all_pred=[]
while time.time() < end_time:
    data = subscriber.recv()
    # Convert received data in bit(4-bytes) to numpy array
    Data_array = np.frombuffer(data, dtype=np.float32).reshape(-1,params['num_channels']+1).T.astype(np.float32)
    
    label= Data_array[-1,:].astype(np.float32)

    Data += [Data_array[:-1,:]]
    
    if len(Data)>= params['step']//64:
        #Data_array=np.expand_dims(Data_array[:-1,:],0)
        Data_raw=np.hstack(Data)
        
        for ch in range(8):
            noise=create_noise(np.min(np.hstack(Data)[ch,:])/max_var,np.max(np.hstack(Data)[ch,:])*max_var,np.hstack(Data).shape[1])
            Data_raw[ch,:]=Data_raw[ch,:]+noise
            
        window_EEG=pp.moving_window(window_EEG,Data_raw,params['step'])   
        Data=[] # reset Data 

        tic=time.time()
        #preprcessing
    
        #psd_data=calculate_psd(window_EEG,params).transpose(1,0)
        
        psd_data=np.zeros([40,8])
        for ch in range(params['num_channels']):
            psd_data[:,ch]=calculate_psd(window_EEG[ch,:],params)

        #psd_data.shape
        psd_2D = buffer_psd(psd_2D, psd_data)
        psd_2D_aux=np.expand_dims(psd_2D.transpose(2,0,1),0)
        
        norm_data=psd_2D_aux.transpose(0, 3, 2, 1)
        #norm_data=pp.NormMinMax_2D(psd_2D_aux,params['norm_axis'],0,1).transpose(0, 3, 2, 1)
        
        prep_time+=[time.time()-tic]
        

        #classification
        tic_model=time.time()

        y_AI_array = np.roll(y_AI_array,-1)

        #model_pred=AI_tf_model.classify_data(np.expand_dims(psd_2D.transpose(1,0,2),0))
        model_pred=AI_tf_model.classify_data(norm_data)

        if np.argmax(model_pred[0]) == 1 and np.argmax(model_pred[1]) != 0 :
        #if np.argmax(model_pred[1]) != 0:
    
            y_AI_array[-1] = np.argmax(model_pred[1][1:])

            AI_time+=[time.time()-tic_model]

            #pos processing 
            y_votting=ppos.majority_voting(y_AI_array, params['num_voting'])
            
            #all_pred+=[y_votting]
            #all_pred+=[y_AI_array[-1]]
            all_pred+=[y_votting.item()+1]
            all_labels+=[label[-1]]
                                           
            # Acc  
            if (all_pred[-1] == int(label[-1])):
                N_correct=N_correct+1
            
            N_interactions=N_interactions+1

    #log results
    if time.time() - update_time_duration>= log_update_time:
        if N_interactions!= 0:

            #reset log time 
            update_time_duration=time.time()
            #comput acc
            Acc=N_correct/N_interactions
            # update params60*
            #print(f'Preprocessing Latency: {round(np.mean(prep_time)*1000,4)}ms | Model Predict Latency: {round(np.mean(AI_time)*1000,4)}ms | Model  accuracy: {Acc}')

            ST.set_model_performance(np.mean(prep_time), np.mean(AI_time),Acc)
            AI_time=[]
            prep_time=[]
            #uncomment to save logs and display it 
            ST.log_info()
            
            N_correct=0
            N_interactions=0
        
print("Média de tempo no pré-processamento",np.array(prep_time).mean() *1000,'ms')
print("Máximo tempo no pré-processamento",np.array(prep_time).max() *1000,'ms')

print("Média de tempo do modelo AI",np.array(AI_time)[10:].mean() *1000,'ms')
print("Máximo tempo do modelo AI",np.array(AI_time)[10:].max() *1000,'ms')

subscriber.close()
context.term()

CPU Usage: 4.8% |CPU Temperature: 0°C% | Memory Usage: 30.5% | Used Disk Space: 8949.913243648 Gbytes | Preprocessing Latency: 1.9737157316925134ms | Model Predict Latency: 31.437103255388607ms | Model  accuracy: 0.3628691983122363 %
CPU Usage: 1.3% |CPU Temperature: 0°C% | Memory Usage: 30.4% | Used Disk Space: 8950.074638336 Gbytes | Preprocessing Latency: 1.860033815323115ms | Model Predict Latency: 29.161373774210613ms | Model  accuracy: 0.36231884057971014 %
CPU Usage: 2.5% |CPU Temperature: 0°C% | Memory Usage: 30.4% | Used Disk Space: 8950.247456768 Gbytes | Preprocessing Latency: 1.9537848750437337ms | Model Predict Latency: 30.46560287475586ms | Model  accuracy: 0.21232876712328766 %
CPU Usage: 3.0% |CPU Temperature: 0°C% | Memory Usage: 30.5% | Used Disk Space: 8950.419718144 Gbytes | Preprocessing Latency: 1.921480927019488ms | Model Predict Latency: 29.655959286390217ms | Model  accuracy: 0.35823429541595925 %
CPU Usage: 51.8% |CPU Temperature: 0°C% | Memory Usage: 31.0% | 